NOTE:
Use this notebook for writeup only. If you want to run some experiments in a notebook, create a separate notebook.

In [ ]:
## Introduction/Motivation
* Why are you solving this problem?
* Why is this project important?

## Problem Description
### Preface
The following ASCII diagram notation is used:
```
snake head:
◄ = left
▲ = up
▼ = down
► = right

F = Food particle
S = highlighted blank square (used to emphasize specific blank squares)

┌───┐
│   │
│   │ = a border around a 3x3 play grid
│   │
└───┘

─│ = "straight" body segments, i.e., adjacent segments are in the same vertical/horizontal plane
┘┐┌└ = "curved" body segments, i.e., adjacent segments form an "L" shape *OR* for the tip of the tail, it may curve to indicate where the end of the tail previously was
```
_NOTE:_
* _body segments and border are composed of same characters_
* _due to differences in font/character rendering, it can create an illusion of "an extra row of space" since vertical space is greater than horizontal space. Select/highlight characters with mouse if unsure._

## Description of Snake Game Model

Snake is a fully-observable, one-player game. The game updates in discrete time steps, the rate of which is controlled either by a framerate parameter, or allowed to update as quickly as the agent provides inputs (for training purposes or to speed up test/evaluation).

The player or agent controls a "snake" character in a 2-dimensional grid. At each discrete timestep the player or agent may take one of two actions, turn left or turn right, or choose to do nothing (continue straight). Stated alternatively, the agent may choose to "move left", "move right", "move up", or "move down" at each timestep. However, it's invalid for a snake to turn 180 degrees; the snake's head would always collide with its body, causing the game to be over. The game engine simply disallows these illegal turns by ignoring inputs that would cause them.

"Food" particles are scattered at random locations throughout the game such that no two food particles may occupy the same location. All food particles have the same value, causing the snake to grow by one grid space.

The semantics of snake growth are such that, on the time step following the intersection of the snake's head and a piece of food, the snakes head proceeds forward one space while the tip of the snakes tail remains fixed in place (for that time step in which the snake is growing.) On time steps in which the snake does not grow, the snakes head advances in the direction indicated by the agent's inputs and the final/tail segment of the snake's body is removed simultaneously (thus, the snake remains fixed in length, but appears to have moved forward such that the body always follows the same path through the game's grid space as the head).

Snake Growth Diagram:
```
1. ┌───┐
   │▲F │
   ││  │ len(snake) == 3
   ││  │
   └───┘
2. ┌───┐
   │┌► │
   ││  │ len (snake) == 3
   │   │
   └───┘
3. ┌───┐
   │┌─►│
   ││  │  len(snake) == 4
   │   │
   └───┘
```

The game is over when either of the two following conditions is met:
1. The snake's head intersects with (occupies the same block as) any block occupied by a segment of its body
```
┌───┐
│┌─┐│
│└─►│ 
│  ││
└───┘
```
2. The snake's head advances outside of the game grid/playing area

```
┌───┐
│┌─┐│
││  │
│└──►
└───┘
```

* Snake Game Objective

The objective is to maximize the length of the snake by taking actions that lead to food, causing the snake to grow, while avoiding actions that cause game over (self-touch, out of bounds).


## Input
### Game Initialization Inputs (problem parameters)
* the game grid dimensions (the number of discrete spatial blocks in the X-dimension and Y-dimension, where snake always moves exactly 1 block at each instant tick of time)
* Framerate, or 0 for "no framerate limit"
  * **❗NOTE: framerate is probably only relevant if we timeout agent decisions during test/evaluation. We currently do not force AI agents to timeout/make a decision by each clock tick, but maybe we should when framerate is non-zero❗**.
* number of food that in play at any given time. After initialization, this is invariant for the duration of play. Whenever a food is eaten, a new one is spawned in any random location where a food does not already exist.
* Snake initial position is currently fixed like this, with length 3, head at top left, pointed down, but ❗it could be paramaterized too❗ (❗in fact if we can read initial position & direction & initial food distribution from a config file/"game save" then we could design training/testing scenarios❗):
```python
snake = Snake(
    segments=[
        Coordinate(0, 0),
        Coordinate(1, 0),
        Coordinate(2, 0),
    ],
    direction=Direction.DOWN,
)

# ┌────────┐
# │┌──────┐│
# ││┌──   ││
# ││▼     ││
# ││      ││
# │└──────┘│
# └────────┘
```

### Gameplay Inputs
* direction for snake to travel 
  * game guards against invalid changes of direction
  * this is effectively equivalent to "two actions: Turn Left, Turn Right" (three actions if you count "Do Not Turn")
* There is additionally a random element: where the food spawns is chosen randomly and not known ahead of time
  * It's random, but with the constraint that two pieces of food cannot occupy the same location simultaneously.
  * ❗The RNG seed could be made into an initialization parameter, and probably should be, to enable repeatable results❗

## Theoretical Best Output

### Snake Length
Snake length is maximized when snake occupies the entire play space + 1. By the pigeon-hole principle, the snakes head must either be out of bounds or occupying the same location as another body segment. In other words, the snake must always lose when `|snake| = grid_area + 1; grid_area = W + H` where `|snake|` is length of snake, `W` is how many discrete blocks wide the game is, and `H` is similarly grid height. Thus, best possible score, if score is length of snake, is `grid_area + 1`.

### Time to Snake Length |N|
Calculating a theoretical best input is tricky due to
1. the random nature of the food spawn
2. multiple food spawn
3. food may spawn "under" snake
4. snake must navigate around its own body to get to food, and body is also moving

However we could state that the "worst-case" round for collecting a piece of food is one in which:

* only one piece of food spawns at any given time
* snake just ate, next piece of food spawns just behind snake's head
* snake currently occupies the entire grid without touching itself

Then, to reach the next piece of food the snake will need to travel the entire distance of the grid, `-1`. This is because if the snake literally traveled the entire distance it would end up in the starting position, we need to go just 1 short of this. In fact, this generalizes to `- |F|` where `|F|` is the number of food that can spawn at any given time because for each piece of food that is spawned, we know 2 food cannot occupy the same location, so each subsequent food is in the next-worst location that maximizes the snake's distance traveled (and thus must be 1 step closer on the snake's optimal path.) However, in such a case, we know the snake eats `F` food if it simply travels the distance of the `grid area -1`, as it will have reached every position that a food occupied when it first set out on its journey and food do not move until eaten. Therefore, the amortized upper-bound time cost per food eaten when there are `|F|` food is `(grid_area - 1)/|F|` ticks.

Thus, an initial estimate on upper-bound of time cost to reach "end game snake length" is `(grid_area + 1) * ((grid_area -1)/|F|)`.

**Conjecture 1**: A snake whose body length is at least `game.grid_width * game.grid_height` can *always* reach the next piece of food within `2*len(snake) + 2` ticks. Thus we would have a worst case time to maximum snake-length:

```python
initial_length = 3
(game.grid_width * game.grid_height - initial_length) * max(
    game.grid_width + game.grid_height,  #1
    min(
        (2 * len(snake)) - len(food) + 1, #2
        (game.grid_width * game.grid_height - 1) / len(food) #3
    )
)
```
Note \#1: The minimum of case \#2 and case \#3 could be less than the corner-to-corner manhattan distance of the game grid. Thus, \#1 accounts for the case where snake length is still small and a piece of food spawns on the opposite side of the game map.

Note \#2: Trivially, we know from above that a snake that is already head-to-tail can reach the next piece of food when that piece of food. However, in that proof, the snake was OK with dying for the last piece. In this case, we may need to create a 'kink' as space permits.

Kink Diagram:
```
1 ┌───────┐ 2┌───────┐ 3┌───────┐
  │┌────┐ │  │┌─────▼│  │┌─────┐│
  │▲┌┐┌┐│ │  ││┌┐┌┐  │  ││┌┐┌S┌┘│
  │F┘└┘└┘ │  │F┘└┘└┘ │  │F┘└┘S◄ │
  └───────┘  └───────┘  └───────┘
```
1. snake just ate, food spawns behind head
2. if snake just follows tail, it will reach food within `|snake|` ticks of diagram 1, but it will grow into/eat its own tail (game over) when it eats the food. Therefore, it travels one extra space instead of immediately following the tail at the fist opportunity
3. snake has completed kink maneuver actions "move right, move down, move left, move down" in the top-right-most 4 cells gave it a 2 block buffer (marked with S) between head and tail.
**CONJECTURE 2**: This is the smallest "buffer" that can be created.

## Intuition
* why should it be better than something we discussed in the class?

## Description
(Be as clear as you can as otherwise we won't understand what you are trying to do.)
* Approach
* Algorithms
* Models (note: Referring to learning/AI models, not the game model. Game model in problem description.)

## Experiments & Results
* List of questions your experiments are designed to answer
* Description of your testbed
* Details of your
   * Experiments
   * Observations
   * Findings

## Discussion
* explain observations (why do we think we got the results that we did?)
* interpret observations (what significance might our results have?)


## Conclusion
* avenues of future work here (threads to pull on that we didn't have time for, interesting results someone might follow up on, etc.)
* briefly summarize (maybe 1 sentence ea?):
  * problem
  * what we tried
  * what we found
  * why what we found matters
  * future work